In [8]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
Data = pd.read_csv("./Result_data.csv")
Data.head()

,EMD_CD,DMCLS_CD_wmode,AGCLS_CD_wmode,frtp_avg,dnst_avg,Area,geometry,mtn_area,Mtn_Ratio,human_adj_area,human_mtn_Ratio,human_Total_Ratio,Fire_Count,ctr_mean,ctr_std,n_ctrs
0,11110101,23,4,0.125,0.6,792617.902967,POLYGON ((953700.0221347539 1954605.0654306228...,67529.485826,0.085198,36117.996122,0.534848,0.045568,0,173.809524,73.940935,21
1,11110102,23,4,0.125,0.6,98821.715201,POLYGON ((953233.4654317541 1953996.9839332262...,59869.101208,0.605829,40495.878201,0.676407,0.409787,0,87.500000,32.274861,4
2,11110103,23,4,0.125,0.6,73271.654659,POLYGON ((953560.2281430522 1954257.4664824517...,776.906691,0.010603,0.000000,0.000000,0.000000,0,50.000000,0.000000,1
3,11110109,23,4,0.125,0.6,258561.672009,POLYGON ((952981.2076348594 1953604.3112467206...,79017.474842,0.305604,77079.210797,0.975470,0.298108,0,160.000000,71.879529,10
4,11110111,23,4,0.125,0.6,417438.349374,"POLYGON ((952769.057857475 1954155.9539321065,...",113433.905865,0.271738,113433.905865,1.000000,0.271738,0,187.500000,84.210131,14


In [ ]:
Features_bin = Data.copy()
Features_bin.Fire_Count = (Features_bin.Fire_Count>0).astype(int)
Target_bin = Features_bin.Fire_Count
# Features_bin.drop(columns=["Fire_Count", "geometry", "EMD_CD"], inplace = True)
# 면적 관련 정보 제거 옵션
# Features_bin.drop(columns=["Fire_Count", "geometry", "EMD_CD", "Area", "mtn_area", "human_adj_area", "n_ctrs"], inplace = True)
Features_bin

,DMCLS_CD_wmode,AGCLS_CD_wmode,frtp_avg,dnst_avg,Mtn_Ratio,human_mtn_Ratio,human_Total_Ratio,ctr_mean,ctr_std
0,23,4,0.125000,0.600000,0.085198,0.534848,0.045568,173.809524,73.940935
1,23,4,0.125000,0.600000,0.605829,0.676407,0.409787,87.500000,32.274861
2,23,4,0.125000,0.600000,0.010603,0.000000,0.000000,50.000000,0.000000
3,23,4,0.125000,0.600000,0.305604,0.975470,0.298108,160.000000,71.879529
4,23,4,0.125000,0.600000,0.271738,1.000000,0.271738,187.500000,84.210131
...,...,...,...,...,...,...,...,...,...
3998,23,5,0.350721,0.838103,0.899488,0.211161,0.189937,660.853909,344.959650
3999,23,4,0.845896,0.829818,0.536337,0.173697,0.093160,97.925764,63.163660
4000,23,5,0.508427,0.828935,0.856081,0.250085,0.214093,369.624682,289.702053
4001,23,4,0.723005,0.791727,0.721694,0.294532,0.212562,199.795082,213.608011


In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    Features_bin, Target_bin, test_size=0.2, stratify=Target_bin, random_state=42)

In [ ]:
xgb = XGBClassifier(scale_pos_weight=3.28,
                    use_label_encoder=False,
                    eval_metric='logloss',
                    random_state=42)

In [20]:
xgb.fit(features_train, target_train)

c:\Users\sc.lee\miniconda3\envs\wildfire_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:34:32] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [21]:
target_pred = xgb.predict(features_test)

In [22]:
print(classification_report(target_test, target_pred))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       614
           1       0.45      0.53      0.49       187

    accuracy                           0.74       801
   macro avg       0.65      0.67      0.66       801
weighted avg       0.76      0.74      0.75       801



In [23]:
print(confusion_matrix(target_test, target_pred))

[[494 120]
 [ 87 100]]


In [24]:
a = pd.Series(xgb.feature_names_in_, xgb.feature_importances_)
a

0.089828       DMCLS_CD_wmode
0.076081       AGCLS_CD_wmode
0.088302             frtp_avg
0.099811             dnst_avg
0.123544            Mtn_Ratio
0.101559      human_mtn_Ratio
0.109190    human_Total_Ratio
0.090781             ctr_mean
0.220904              ctr_std
dtype: object